In [4]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [5]:
# !wget https://f000.backblazeb2.com/file/malaya-speech-model/pretrained/output-large-conformer-v4.tar.gz
# !tar -zxf output-large-conformer-v4.tar.gz
# !rm  output-large-conformer-v4.tar.gz

In [6]:
import malaya_speech.train.model.conformer as conformer
import malaya_speech.train.model.transducer as transducer
import malaya_speech
import tensorflow as tf
import numpy as np
np.set_printoptions(threshold=1)

In [7]:
subwords = malaya_speech.subword.load('transducer.subword')
vocabs = {i: subwords._id_to_subword(i) for i in range(subwords.vocab_size - 1)}
vocabs[-1] = ''

In [8]:
encoded = subwords.encode('saya suka ayam')
[vocabs[e - 1] for e in encoded], encoded

(['say', 'a_', 'suk', 'a_', 'aya', b'm'], [33, 1, 468, 1, 138, 883])

In [9]:
featurizer = malaya_speech.tf_featurization.STTFeaturizer(
    normalize_per_feature = True
)

In [10]:
X = tf.compat.v1.placeholder(tf.float32, [None, None], name = 'X_placeholder')
X_len = tf.compat.v1.placeholder(tf.int32, [None], name = 'X_len_placeholder')

In [11]:
batch_size = tf.shape(X)[0]
features = tf.TensorArray(dtype = tf.float32, size = batch_size, dynamic_size = True, infer_shape = False)
features_len = tf.TensorArray(dtype = tf.int32, size = batch_size)

init_state = (0, features, features_len)

def condition(i, features, features_len):
    return i < batch_size

def body(i, features, features_len):
    f = featurizer(X[i, :X_len[i]])
    f_len = tf.shape(f)[0]
    return i + 1, features.write(i, f), features_len.write(i, f_len)

_, features, features_len = tf.while_loop(condition, body, init_state)
features_len = features_len.stack()
padded_features = tf.TensorArray(dtype = tf.float32, size = batch_size)
padded_lens = tf.TensorArray(dtype = tf.int32, size = batch_size)
maxlen = tf.reduce_max(features_len)

init_state = (0, padded_features, padded_lens)

def condition(i, padded_features, padded_lens):
    return i < batch_size

def body(i, padded_features, padded_lens):
    f = features.read(i)
    len_f = tf.shape(f)[0]
    f = tf.pad(f, [[0, maxlen - tf.shape(f)[0]], [0,0]])
    return i + 1, padded_features.write(i, f), padded_lens.write(i, len_f)

_, padded_features, padded_lens = tf.while_loop(condition, body, init_state)
padded_features = padded_features.stack()
padded_lens = padded_lens.stack()
padded_lens.set_shape((None,))
padded_features.set_shape((None, None, 80))
padded_features = tf.expand_dims(padded_features, -1)
padded_features, padded_lens

(<tf.Tensor 'ExpandDims:0' shape=(?, ?, 80, 1) dtype=float32>,
 <tf.Tensor 'TensorArrayStack_2/TensorArrayGatherV3:0' shape=(?,) dtype=int32>)

In [12]:
padded_features = tf.identity(padded_features, name = 'padded_features')
padded_lens = tf.identity(padded_lens, name = 'padded_lens')

In [13]:
config = malaya_speech.config.conformer_large_encoder_config
config['dropout'] = 0.0
conformer_model = conformer.Model(**config)
decoder_config = malaya_speech.config.conformer_large_decoder_config
decoder_config['embed_dropout'] = 0.0
transducer_model = transducer.rnn.Model(
    conformer_model, vocabulary_size = subwords.vocab_size, **decoder_config
)

In [14]:
p = tf.compat.v1.placeholder(tf.int32, [None, None])
z = tf.zeros((tf.shape(p)[0], 1),dtype=tf.int32)
c = tf.concat([z, p], axis = 1)
p_len = tf.compat.v1.placeholder(tf.int32, [None])
c

<tf.Tensor 'concat:0' shape=(?, ?) dtype=int32>

In [15]:
training = True

In [16]:
logits = transducer_model([padded_features, c, p_len], training = training)
logits

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


<tf.Tensor 'transducer/transducer_joint/transducer_joint_vocab/BiasAdd:0' shape=(?, ?, ?, 1030) dtype=float32>

In [17]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [18]:
var_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
saver = tf.train.Saver(var_list = var_list)
saver.restore(sess, 'output-large-conformer/model.ckpt')

INFO:tensorflow:Restoring parameters from output-large-conformer/model.ckpt


In [19]:
decoded = transducer_model.greedy_decoder(padded_features, padded_lens, training = training)
decoded

(<tf.Tensor 'while_2/Exit_1:0' shape=(?, ?) dtype=int32>,
 <tf.Tensor 'while_2/Exit_2:0' shape=(?, ?, 1030) dtype=float32>)

In [20]:
decoded_logits = tf.identity(decoded[1], name = 'greedy_decoder_logits')
decoded = tf.identity(decoded[0], name = 'greedy_decoder')
decoded

<tf.Tensor 'greedy_decoder:0' shape=(?, ?) dtype=int32>

In [21]:
encoded = transducer_model.encoder(padded_features, training = training)
encoded = tf.identity(encoded, name = 'encoded')
encoded_placeholder = tf.placeholder(tf.float32, [config['dmodel']], name = 'encoded_placeholder')
predicted_placeholder = tf.placeholder(tf.int32, None, name = 'predicted_placeholder')
t = transducer_model.predict_net.get_initial_state().shape
states_placeholder = tf.placeholder(tf.float32, [int(i) for i in t], name = 'states_placeholder')

ytu, new_states = transducer_model.decoder_inference(
    encoded=encoded_placeholder,
    predicted=predicted_placeholder,
    states=states_placeholder,
    training = training,
)

ytu = tf.identity(ytu, name = 'ytu')
new_states = tf.identity(new_states, name = 'new_states')
ytu, new_states

(<tf.Tensor 'ytu:0' shape=(1030,) dtype=float32>,
 <tf.Tensor 'new_states:0' shape=(1, 2, 1, 640) dtype=float32>)

In [22]:
initial_states = transducer_model.predict_net.get_initial_state()
initial_states = tf.identity(initial_states, name = 'initial_states')

In [23]:
# sess = tf.Session()
# sess.run(tf.global_variables_initializer())

In [24]:
# var_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
# saver = tf.train.Saver(var_list = var_list)
# saver.restore(sess, 'asr-small-conformer-transducer/model.ckpt-325000')

In [25]:
files = [
    'speech/record/savewav_2020-11-26_22-36-06_294832.wav',
    'speech/record/savewav_2020-11-26_22-40-56_929661.wav',
    'speech/record/675.wav',
    'speech/record/664.wav',
    'speech/example-speaker/husein-zolkepli.wav',
    'speech/example-speaker/mas-aisyah.wav',
    'speech/example-speaker/khalil-nooh.wav',
    'speech/example-speaker/shafiqah-idayu.wav',
    'speech/khutbah/wadi-annuar.wav',
]
front_pad = 200
back_pad = 2000
inputs = [malaya_speech.load(f)[0] for f in files]
padded, lens = malaya_speech.padding.sequence_1d(inputs, return_len = True)
back = np.zeros(shape = (len(inputs), back_pad))
front = np.zeros(shape = (len(inputs), front_pad))
padded = np.concatenate([front, padded, back], axis = -1)
lens = [l + front_pad + back_pad for l in lens]

In [26]:
%%time

r = sess.run(decoded, feed_dict = {X: padded, X_len: lens})
for row in r:
    print(malaya_speech.subword.decode(subwords, row[row > 0]))

hello nama saya send saya tak suka mandi ketat saya masam
hello nama saya husin saya suka mandi saya mandi setiap hari
ini dan melalui kenyataan mesej itu mastura menegaskan
pilihan tepat apabila dia kini lebih berani dan
testing nama saya husin bin zulkifli
sebut perkataan angka
tolong sebut antikata
nama saya syafiqah idayu
jadi dalam perjalanan ini dunia yang susah ini ketika nabi mengajar muadz bin jabal tadi ini allah maha ini
CPU times: user 41.1 s, sys: 9.27 s, total: 50.4 s
Wall time: 6.79 s


In [27]:
%%time

greedy, padded_lens_ = sess.run([decoded_logits, padded_lens], feed_dict = {X: padded, X_len: lens})
padded_lens_ = padded_lens_ // conformer_model.conv_subsampling.time_reduction_factor

CPU times: user 49.1 s, sys: 3.75 s, total: 52.8 s
Wall time: 5.45 s


In [28]:
# from ctc_decoders import Scorer
# from ctc_decoders import ctc_beam_search_decoder

In [29]:
# unique_vocab = [subwords._id_to_subword(i) for i in range(subwords.vocab_size - 1)]
# unique_vocab = [w.decode(encoding = 'ISO-8859-1') if isinstance(w, bytes) else w for w in unique_vocab]
# unique_vocab = [w.replace('_', ' ') for w in unique_vocab]
# len(unique_vocab)

In [30]:
# scorer = Scorer(0.02, 1.0, 'out.trie.klm', unique_vocab)

In [31]:
# l = greedy[0,:padded_lens_[0]]
# l = np.concatenate([l[:,1:], l[:,:1]], axis = 1)
# l.shape

In [32]:
%%time

encoded_, padded_lens_  = sess.run([encoded, padded_lens], feed_dict = {X: padded, X_len: lens})
padded_lens_ = padded_lens_ // conformer_model.conv_subsampling.time_reduction_factor
s = sess.run(initial_states)

CPU times: user 26.2 s, sys: 901 ms, total: 27.1 s
Wall time: 4.77 s


In [33]:
import collections
import numpy as np

BeamHypothesis = collections.namedtuple(
    'BeamHypothesis', ('score', 'prediction', 'states')
)

def apply_temp(logits_BxN, temperature):
    uniform_noise_BxN = np.random.uniform(size = logits_BxN.shape)
    logits_BxN += -np.log(-np.log(uniform_noise_BxN)) * temperature
    return logits_BxN

def transducer(
    enc,
    total,
    initial_states,
    encoded_placeholder,
    predicted_placeholder,
    states_placeholder,
    ytu,
    new_states,
    sess,
    beam_width = 10,
    temperature = 0.0,
    score_norm = True,
):
    kept_hyps = [
        BeamHypothesis(score = 0.0, prediction = [0], states = initial_states)
    ]
    B = kept_hyps
    for i in range(total):
        A = B
        B = []
        while True:
            y_hat = max(A, key = lambda x: x.score)
            A.remove(y_hat)
            ytu_, new_states_ = sess.run(
                [ytu, new_states],
                feed_dict = {
                    encoded_placeholder: enc[i],
                    predicted_placeholder: y_hat.prediction[-1],
                    states_placeholder: y_hat.states,
                },
            )
            if temperature > 0:
                ytu_ = apply_temp(ytu_, temperature = temperature)
            top_k = ytu_[1:].argsort()[-beam_width:][::-1]
            B.append(BeamHypothesis(
                score = y_hat.score + ytu_[0],
                prediction = y_hat.prediction,
                states = y_hat.states,
            ))
            for k in top_k:
                
                beam_hyp = BeamHypothesis(
                    score = y_hat.score + ytu_[k + 1],
                    prediction = y_hat.prediction + [k + 1],
                    states = new_states_,
                )
                A.append(beam_hyp)
            
            hyps_max = max(A, key=lambda x: x.score).score
            kept_most_prob = sorted(
                [hyp for hyp in B if hyp.score > hyps_max],
                key=lambda x: x.score,
            )
            if len(kept_most_prob) >= beam_width:
                B = kept_most_prob
                break
    
    if score_norm:
        B.sort(key=lambda x: x.score / len(x.prediction), reverse=True)
    else:
        B.sort(key=lambda x: x.score, reverse=True)
    return B[0].prediction

In [34]:
# i = 0
# o = transducer(
#     enc = encoded_[i],
#     total = padded_lens_[i],
#     initial_states = s,
#     encoded_placeholder = encoded_placeholder,
#     predicted_placeholder = predicted_placeholder,
#     states_placeholder = states_placeholder,
#     ytu = ytu,
#     new_states = new_states,
#     sess = sess,
#     beam_width = 2,
# )

In [35]:
# malaya_speech.subword.decode(subwords, o[1:])

In [36]:
%%time

for i in range(len(encoded_)):
    r = transducer(
        enc = encoded_[i],
        total = padded_lens_[i],
        initial_states = s,
        encoded_placeholder = encoded_placeholder,
        predicted_placeholder = predicted_placeholder,
        states_placeholder = states_placeholder,
        ytu = ytu,
        new_states = new_states,
        sess = sess,
        beam_width = 10,
    )
    print(malaya_speech.subword.decode(subwords, r))

hello nama saya mesin saya tak suka mandi ketat saya memasang
helo nama saya husin saya suka mandi saya mandi setiap hari
ini dan melalui kenyataan mesej itu mastura menegaskan
pilihan tepat apabila dia kini lebih berani dan
testing nama saya husin bin zulkifli
sebut perkataan angka
tolong sebut antikata
nama saya syafiqah idayu
jadi dalam perjalanan ini dunia yang susah ini ketika nabi mengajar muadz bin jabal tadi ini allah maha ini
CPU times: user 2min 14s, sys: 14.9 s, total: 2min 29s
Wall time: 33 s


In [ ]:
# %%time

# for i in range(len(encoded_)):
#     r = transducer(
#         enc = encoded_[i],
#         total = padded_lens_[i],
#         initial_states = s,
#         encoded_placeholder = encoded_placeholder,
#         predicted_placeholder = predicted_placeholder,
#         states_placeholder = states_placeholder,
#         ytu = ytu,
#         new_states = new_states,
#         sess = sess,
#         beam_width = 10,
#         temperature = 0.1
#     )
#     print(malaya_speech.subword.decode(subwords, r))

In [ ]:
# !wget https://f000.backblazeb2.com/file/malaya-speech-model/language-model/redape-community/model.trie.klm

In [ ]:
import kenlm

kenlm_model = kenlm.Model('out.trie.klm')

In [ ]:
from pyctcdecode.language_model import LanguageModel

language_model = LanguageModel(kenlm_model, alpha = 0.01, beta = 0.5)

In [ ]:
import collections
import numpy as np
import tensorflow as tf
import re
import heapq
from typing import Any, Collection, Dict, Iterable, List, Optional, Tuple, Union

LMState = Optional[Union[kenlm.State, List[kenlm.State]]]

BeamHypothesis_LM = collections.namedtuple(
    'BeamHypothesis_LM', ('score', 'prediction', 'states', 'text', 'next_word', 'word_part')
)

def _sort_and_trim_beams(beams: list, beam_width: int) -> list:
    return heapq.nlargest(beam_width, beams, key=lambda x: x.score)

def prune_space(string):
    return re.sub(r'[ ]+', ' ', string).strip()

def _merge_tokens(token_1, token_2) -> str:
    if len(token_2) == 0:
        text = token_1
    elif len(token_1) == 0:
        text = token_2
    else:
        text = token_1 + " " + token_2
    return prune_space(text)

def _prune_history(beams, lm_order: int):
    """Filter out beams that are the same over max_ngram history.
    Since n-gram language models have a finite history when scoring a new token, we can use that
    fact to prune beams that only differ early on (more than n tokens in the past) and keep only the
    higher scoring ones. Note that this helps speed up the decoding process but comes at the cost of
    some amount of beam diversity. If more than the top beam is used in the output it should
    potentially be disabled.
    """
    # let's keep at least 1 word of history
    min_n_history = max(1, lm_order - 1)
    seen_hashes = set()
    filtered_beams = []
    # for each beam after this, check if we need to add it
    for beam in beams:
        text = beam.text
        next_word = beam.next_word
        word_part = beam.word_part
        last_char = beam.prediction[-1]
        logit_score = beam.score
        hash_idx = (tuple(text.split()[-min_n_history:]), word_part, last_char)
        if hash_idx not in seen_hashes:
            beam_hyp = BeamHypothesis_LM(
                score = logit_score,
                prediction = beam.prediction,
                states = beam.states,
                text = text,
                next_word = next_word,
                word_part = word_part
            )
            filtered_beams.append(beam_hyp)
            seen_hashes.add(hash_idx)
    return filtered_beams

def get_lm_beams(beams, cached_lm_scores,
        cached_partial_token_scores,
        is_eos: bool = False):
    
    new_beams = []
    for beam in beams:
        text = beam.text
        next_word = beam.next_word
        word_part = beam.word_part
        last_char = beam.prediction[-1]
        logit_score = beam.score
        new_text = _merge_tokens(text, next_word)
        if new_text not in cached_lm_scores:
            _, prev_raw_lm_score, start_state = cached_lm_scores[text]
            score, end_state = language_model.score(start_state, next_word, is_last_word=is_eos)
            raw_lm_score = prev_raw_lm_score + score
            lm_hw_score = raw_lm_score
            cached_lm_scores[new_text] = (lm_hw_score, raw_lm_score, end_state)
        lm_score, _, _ = cached_lm_scores[new_text]
        if len(word_part) > 0:
            if word_part not in cached_partial_token_scores:
                cached_partial_token_scores[word_part] = language_model.score_partial_token(
                    word_part
                )
            lm_score += cached_partial_token_scores[word_part]
        beam_hyp = BeamHypothesis_LM(
            score = logit_score + lm_score,
            prediction = beam.prediction,
            states = beam.states,
            text = new_text,
            next_word = '',
            word_part = word_part
        )
        new_beams.append(beam_hyp)

    return new_beams
            
        
def transducer_lm(
    enc,
    total,
    initial_states,
    encoded_placeholder,
    predicted_placeholder,
    states_placeholder,
    ytu,
    new_states,
    sess,
    beam_width = 10,
    token_min_logp = -20.0,
    beam_prune_logp = -1.0,
    temperature = 0.0,
):
    kept_hyps = [
        BeamHypothesis_LM(score = 0.0, prediction = [0], states = initial_states, text = '', next_word = '',
                      word_part = '')
    ]
    start_state = kenlm.State()
    cached_lm_scores: Dict[str, Tuple[float, float, LMState]] = {
        '': (0.0, 0.0, language_model.get_start_state())
    }
    cached_p_lm_scores: Dict[str, float] = {}
    B = kept_hyps
    for i in range(total):
        A = B
        B = []
        while True:
            y_hat = max(A, key = lambda x: x.score)
            A.remove(y_hat)
            ytu_, new_states_ = sess.run(
                [ytu, new_states],
                feed_dict = {
                    encoded_placeholder: enc[i],
                    predicted_placeholder: y_hat.prediction[-1],
                    states_placeholder: y_hat.states,
                },
            )
            ytu_ = np.log(ytu_)
            if temperature > 0:
                ytu_ = apply_temp(ytu_, temperature=temperature)
            # ytu_ = np.clip(ytu_, np.log(1e-15), 0)
            B.append(BeamHypothesis_LM(
                score=y_hat.score + ytu_[0],
                prediction=y_hat.prediction,
                states=y_hat.states,
                text=y_hat.text,
                next_word=y_hat.next_word,
                word_part=y_hat.word_part,
            ))
            ytu_ = ytu_[1:]
            max_idx = ytu_.argmax()
            idx_list = set(np.where(ytu_ >= token_min_logp)[0]) | {max_idx}
            for k in idx_list:
                w = vocabs[k]
                if isinstance(w, bytes):
                    w = w.decode(encoding = 'ISO-8859-1')
                w = w.replace('_', ' ')
                s = y_hat.score + ytu_[k]
                p = y_hat.prediction + [k + 1]
            
                if w[-1] == ' ':
                    beam_hyp = BeamHypothesis_LM(
                        score = s,
                        prediction = p,
                        states = new_states_,
                        text = y_hat.text,
                        next_word = y_hat.word_part + w,
                        word_part = ''
                    )
                else:
                    beam_hyp = BeamHypothesis_LM(
                        score = s,
                        prediction = p,
                        states = new_states_,
                        text = y_hat.text,
                        next_word = y_hat.next_word,
                        word_part = y_hat.word_part + w
                    )
                A.append(beam_hyp)
            
            A = get_lm_beams(A, cached_lm_scores, cached_p_lm_scores)
            max_A = max(A, key = lambda x: x.score)
            max_score = max_A.score
            A = [b for b in A if b.score >= max_score + beam_prune_logp]
            A = _sort_and_trim_beams(A, beam_width = beam_width)
            A = _prune_history(A, lm_order=language_model.order)
            
            max_A = max(A, key=lambda x: x.score)
            hyps_max = max_A.score
            kept_most_prob = sorted(
                [hyp for hyp in B if hyp.score > hyps_max],
                key=lambda x: x.score,
            )
            if len(kept_most_prob) > 0:
                B = kept_most_prob
                break
            

    new_beams = []
    for beam in B:
        text = beam.text
        next_word = beam.next_word
        word_part = beam.word_part
        last_char = beam.prediction[-1]
        logit_score = beam.score
        beam_hyp = BeamHypothesis_LM(
            score = logit_score,
            prediction = beam.prediction,
            states = beam.states,
            text = text,
            next_word = word_part,
            word_part = ''
        )
        new_beams.append(beam_hyp)
    A = get_lm_beams(new_beams, cached_lm_scores, cached_p_lm_scores,is_eos=True)
    max_A = max(A, key = lambda x: x.score)
    max_score = max_A.score
    A = [b for b in A if b.score >= max_score + beam_prune_logp]
    A = _sort_and_trim_beams(A, beam_width = beam_width)
    return A[0].prediction

In [ ]:
language_model = LanguageModel(kenlm_model, alpha = 0.03, beta = 1.0)

In [ ]:
i = 8
o = transducer_lm(
    enc = encoded_[i],
    total = padded_lens_[i],
    initial_states = s,
    encoded_placeholder = encoded_placeholder,
    predicted_placeholder = predicted_placeholder,
    states_placeholder = states_placeholder,
    ytu = ytu,
    new_states = new_states,
    sess = sess,
    beam_width = 3,
)
malaya_speech.subword.decode(subwords, o)

In [ ]:
%%time

encoded_, padded_lens_  = sess.run([encoded, padded_lens], feed_dict = {X: padded, X_len: lens})
padded_lens_ = padded_lens_ // conformer_model.conv_subsampling.time_reduction_factor
s = sess.run(initial_states)

for i in range(len(encoded_)):
    r = transducer(
        enc = encoded_[i],
        total = padded_lens_[i],
        initial_states = s,
        encoded_placeholder = encoded_placeholder,
        predicted_placeholder = predicted_placeholder,
        states_placeholder = states_placeholder,
        ytu = ytu,
        new_states = new_states,
        sess = sess,
        beam_width = 1,
    )

    print(malaya_speech.subword.decode(subwords, r))

In [37]:
l = padded_lens // transducer_model.encoder.conv_subsampling.time_reduction_factor
encoded = transducer_model.encoder(padded_features, training = training)
g = transducer_model._perform_greedy(encoded[0], l[0],
                                tf.constant(0, dtype = tf.int32),
                                transducer_model.predict_net.get_initial_state())
g

Hypothesis(index=<tf.Tensor 'while_3/Exit_1:0' shape=() dtype=int32>, prediction=<tf.Tensor 'TensorArrayStack_3/TensorArrayGatherV3:0' shape=(?,) dtype=int32>, states=<tf.Tensor 'while_3/Exit_3:0' shape=(1, 2, 1, 640) dtype=float32>, alignment=<tf.Tensor 'TensorArrayStack_4/TensorArrayGatherV3:0' shape=(?, 1030) dtype=float32>)

In [38]:
indices = g.prediction
minus_one = -1 * tf.ones_like(indices, dtype=tf.int32)
blank_like = 0 * tf.ones_like(indices, dtype=tf.int32)
indices = tf.where(indices == minus_one, blank_like, indices)
num_samples = tf.cast(X_len[0], dtype=tf.float32)
total_time_reduction_factor = featurizer.frame_step
stime = tf.range(0, num_samples, delta=total_time_reduction_factor, dtype=tf.float32)
stime /= tf.cast(featurizer.sample_rate, dtype=tf.float32)
stime = stime[::tf.shape(stime)[0] // tf.shape(indices)[0]]
stime.set_shape((None,))
non_blank = tf.where(tf.not_equal(indices, 0))
non_blank_transcript = tf.gather_nd(indices, non_blank)
non_blank_stime = tf.gather_nd(stime, non_blank)
non_blank_transcript = tf.identity(non_blank_transcript, name = 'non_blank_transcript')
non_blank_stime = tf.identity(non_blank_stime, name = 'non_blank_stime')

In [39]:
%%time

r = sess.run([non_blank_transcript, non_blank_stime], feed_dict = {X: padded, X_len: lens})

CPU times: user 29.1 s, sys: 1.5 s, total: 30.6 s
Wall time: 5.9 s


In [40]:
words, indices = [], []
for no, ids in enumerate(r[0]):
    w = subwords._id_to_subword(ids - 1)
    if type(w) == bytes:
        w = w.decode()
    words.extend([w, None])
    indices.extend([no, None])

In [41]:
import six
from malaya_speech.utils import text_encoder

def _trim_underscore_and_tell(token):
    if token.endswith('_'):
        return token[:-1], True
    return token, False

def decode(ids):
    ids = text_encoder.pad_decr(ids)
    subword_ids = ids
    del ids

    subwords_ = []
    prev_bytes = []
    prev_ids = []
    ids = []

    def consume_prev_bytes():
        if prev_bytes:
            subwords_.extend(prev_bytes)
            ids.extend(prev_ids)
        return [], []

    for no, subword_id in enumerate(subword_ids):
        subword = subwords._id_to_subword(subword_id)
        if isinstance(subword, six.binary_type):
            # Byte-encoded
            prev_bytes.append(subword.decode('utf-8', 'replace'))
            if subword == b' ':
                prev_ids.append(None)
            else:
                prev_ids.append(no)
        else:
            # If there were bytes previously, convert to unicode.
            prev_bytes, prev_ids = consume_prev_bytes()
            trimmed, add_space = _trim_underscore_and_tell(subword)
            ids.append(no)
            subwords_.append(trimmed)
            if add_space:
                subwords_.append(' ')
                ids.append(None)
    prev_bytes = consume_prev_bytes()

    return subwords_, ids

words, indices = decode(r[0])

In [42]:
def combined_indices(subwords, ids, l, reduction_factor = 160, sample_rate = 16000):
    result, temp_l, temp_r = [], [], []
    for i in range(len(subwords)):
        if ids[i] is not None:
            temp_l.append(subwords[i])
            temp_r.append(l[ids[i]])
        else:
            data = {'text': ''.join(temp_l), 
                    'start': round(temp_r[0],4), 
                    'end': round(temp_r[-1] + (reduction_factor / sample_rate), 4)}
            result.append(data)
            temp_l, temp_r = [], []
    
    if len(temp_l):
        data = {'text': ''.join(temp_l), 
                'start': round(temp_r[0],4), 
                'end': round(temp_r[-1] + (reduction_factor / sample_rate), 4)}
        result.append(data)
    
    return result

In [43]:
combined_indices(words, indices, r[1])

[{'text': 'hello', 'start': 0.0, 'end': 0.13},
 {'text': 'nama', 'start': 0.76, 'end': 0.97},
 {'text': 'saya', 'start': 1.0, 'end': 1.25},
 {'text': 'send', 'start': 1.6, 'end': 1.81},
 {'text': 'saya', 'start': 2.64, 'end': 2.73},
 {'text': 'tak', 'start': 2.8, 'end': 2.81},
 {'text': 'suka', 'start': 2.92, 'end': 3.09},
 {'text': 'mandi', 'start': 3.16, 'end': 3.37},
 {'text': 'ketat', 'start': 4.56, 'end': 4.73},
 {'text': 'saya', 'start': 4.84, 'end': 5.01},
 {'text': 'masam', 'start': 5.24, 'end': 5.53}]

In [44]:
saver = tf.train.Saver()
saver.save(sess, 'output-large-conformer-v2/model.ckpt')

'output-large-conformer-v2/model.ckpt'

In [45]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'gather' in n.op.lower()
        or 'placeholder' in n.name
        or 'encoded' in n.name
        or 'decoder' in n.name
        or 'ytu' in n.name
        or 'new_states' in n.name
        or 'padded_' in n.name
        or 'initial_states' in n.name
        or 'non_blank' in n.name)
        and 'adam' not in n.name
        and 'global_step' not in n.name
        and 'Assign' not in n.name
        and 'ReadVariableOp' not in n.name
        and 'Gather' not in n.name
    ]
)
strings.split(',')

['X_placeholder',
 'X_len_placeholder',
 'padded_features',
 'padded_lens',
 'transducer/transducer_prediction/transducer_prediction_embedding/embeddings',
 'greedy_decoder_logits',
 'greedy_decoder',
 'encoded',
 'encoded_placeholder',
 'predicted_placeholder',
 'states_placeholder',
 'ytu',
 'new_states',
 'initial_states',
 'non_blank_transcript',
 'non_blank_stime']

In [46]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [47]:
freeze_graph('output-large-conformer-v2', strings)

INFO:tensorflow:Restoring parameters from output-large-conformer-v2/model.ckpt
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 561 variables.
INFO:tensorflow:Converted 561 variables to const ops.
27434 ops in the final graph.


In [48]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
                
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
        
    return graph

In [ ]:
g = load_graph('output-large-conformer/frozen_model.pb')

In [ ]:
input_nodes = [
    'X_placeholder',
    'X_len_placeholder',
    'encoded_placeholder',
    'predicted_placeholder',
    'states_placeholder',
]
output_nodes = [
    'greedy_decoder',
    'encoded',
    'ytu',
    'new_states',
    'padded_features',
    'padded_lens',
    'initial_states',
    'non_blank_transcript',
    'non_blank_stime'
]
inputs = {n: g.get_tensor_by_name(f'import/{n}:0') for n in input_nodes}
outputs = {n: g.get_tensor_by_name(f'import/{n}:0') for n in output_nodes}

In [ ]:
test_sess = tf.Session(graph = g)

In [ ]:
r = test_sess.run(outputs['greedy_decoder'], feed_dict = {inputs['X_placeholder']: padded, 
                                                          inputs['X_len_placeholder']: lens})

In [ ]:
for row in r:
    print(malaya_speech.subword.decode(subwords, row[row > 0]))

In [ ]:
encoded_, padded_lens_, s  = test_sess.run([outputs['encoded'], outputs['padded_lens'], outputs['initial_states']], 
                                        feed_dict = {inputs['X_placeholder']: padded, 
                                                     inputs['X_len_placeholder']: lens})

padded_lens_ = padded_lens_ // conformer_model.conv_subsampling.time_reduction_factor

In [ ]:
i = 0
r = transducer(
    enc = encoded_[i],
    total = padded_lens_[i],
    initial_states = s,
    encoded_placeholder = inputs['encoded_placeholder'],
    predicted_placeholder = inputs['predicted_placeholder'],
    states_placeholder = inputs['states_placeholder'],
    ytu = outputs['ytu'],
    new_states = outputs['new_states'],
    sess = test_sess,
    beam_width = 1,
)

malaya_speech.subword.decode(subwords, r)

In [49]:
from tensorflow.tools.graph_transforms import TransformGraph

In [50]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-10, fallback_max=10)',
             'strip_unused_nodes',
             'sort_by_execution_order']

input_nodes = [
    'X_placeholder',
    'X_len_placeholder',
    'encoded_placeholder',
    'predicted_placeholder',
    'states_placeholder',
]
output_nodes = [
    'greedy_decoder',
    'encoded',
    'ytu',
    'new_states',
    'padded_features',
    'padded_lens',
    'initial_states',
    'non_blank_transcript',
    'non_blank_stime'
]

pb = 'output-large-conformer-v2/frozen_model.pb'

input_graph_def = tf.GraphDef()
with tf.gfile.FastGFile(pb, 'rb') as f:
    input_graph_def.ParseFromString(f.read())

transformed_graph_def = TransformGraph(input_graph_def, 
                                           input_nodes,
                                           output_nodes, transforms)
    
with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
    f.write(transformed_graph_def.SerializeToString())

Instructions for updating:
Use tf.gfile.GFile.


In [51]:
# g = load_graph('output-base-conformer/frozen_model.pb.quantized')

In [ ]:
inputs = {n: g.get_tensor_by_name(f'import/{n}:0') for n in input_nodes}
outputs = {n: g.get_tensor_by_name(f'import/{n}:0') for n in output_nodes}
test_sess = tf.Session(graph = g)

In [ ]:
r = test_sess.run(outputs['greedy_decoder'], feed_dict = {inputs['X_placeholder']: padded, 
                                                          inputs['X_len_placeholder']: lens})

In [ ]:
for row in r:
    print(malaya_speech.subword.decode(subwords, row[row > 0]))

In [ ]:
encoded_, padded_lens_, s  = test_sess.run([outputs['encoded'], outputs['padded_lens'], outputs['initial_states']], 
                                        feed_dict = {inputs['X_placeholder']: padded, 
                                                     inputs['X_len_placeholder']: lens})

padded_lens_ = padded_lens_ // conformer_model.conv_subsampling.time_reduction_factor

In [ ]:
# i = 0
# r = transducer(
#     enc = encoded_[i],
#     total = padded_lens_[i],
#     initial_states = s,
#     encoded_placeholder = inputs['encoded_placeholder'],
#     predicted_placeholder = inputs['predicted_placeholder'],
#     states_placeholder = inputs['states_placeholder'],
#     ytu = outputs['ytu'],
#     new_states = outputs['new_states'],
#     sess = test_sess,
#     beam_width = 1,
# )

# malaya_speech.subword.decode(subwords, r)

In [52]:
b2_application_key_id = os.environ['b2_application_key_id']
b2_application_key = os.environ['b2_application_key']

In [53]:
from b2sdk.v1 import *
info = InMemoryAccountInfo()
b2_api = B2Api(info)
application_key_id = b2_application_key_id
application_key = b2_application_key
b2_api.authorize_account("production", application_key_id, application_key)
file_info = {'how': 'good-file'}
b2_bucket = b2_api.get_bucket_by_name('malaya-speech-model')

In [54]:
directory = 'output-large-conformer-v2'
tar = 'output-large-conformer-v4-v2.tar.gz'
os.system(f'tar -czvf {tar} {directory}')

0

In [55]:
outPutname = f'pretrained/{tar}'
b2_bucket.upload_local_file(
    local_file=tar,
    file_name=outPutname,
    file_infos=file_info,
)

In [56]:
!rm {tar}

In [57]:
file = 'output-large-conformer-v2/frozen_model.pb'
outPutname = 'speech-to-text-transducer/large-conformer/model.pb'
b2_bucket.upload_local_file(
    local_file=file,
    file_name=outPutname,
    file_infos=file_info,
)

In [58]:
file = 'output-large-conformer-v2/frozen_model.pb.quantized'
outPutname = 'speech-to-text-transducer/large-conformer-quantized/model.pb'
b2_bucket.upload_local_file(
    local_file=file,
    file_name=outPutname,
    file_infos=file_info,
)

In [59]:
!rm -rf output-large-conformer-v2 output-large-conformer-v4-v2.tar.gz